# Installations

In [ ]:
%%writefile requirements.txt
optuna
xgboost
lightgbm
scikit-learn
optuna_fast_fanova
mlflow

In [ ]:
!pip install -qU -r requirements.txt

# Imports

In [ ]:
# Standard Library Imports
import gc
import json
import os
import sys
from datetime import datetime
from pathlib import Path

# Third-Party Imports
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import mlflow
import mlflow.lightgbm
import mlflow.sklearn
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
import statsmodels.api as sm
from lightgbm import LGBMRegressor
from optuna.importance import get_param_importances
from optuna.pruners import HyperbandPruner
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances
)
from optuna_fast_fanova import FanovaImportanceEvaluator
from scipy.stats import mstats
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import (
    BaggingRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    RandomForestRegressor,
    StackingRegressor,
    VotingRegressor
)
from sklearn.inspection import PartialDependenceDisplay
from sklearn.linear_model import (
    ElasticNet,
    HuberRegressor,
    Lasso,
    LassoLarsCV,
    LinearRegression,
    Ridge
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import (
    RepeatedKFold,
    learning_curve,
    train_test_split
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import all_estimators
from xgboost import XGBRegressor
import yaml

# Local Imports
from mlflow.tracking import MlflowClient

# Constants

In [ ]:
config_yaml_path = os.path.join(
    '/content',
    'drive',
    'MyDrive',
    'Colab',
    'Machine Learning',
    'Regression',
    'Radancy',
    'config.yaml'
)

# Load config
with open(config_yaml_path, "r") as file:
  config = yaml.safe_load(file)

# Cross-platform path assembly
PATH = Path().joinpath(*config["PATH_PARTS"])

# Other parameters
RANDOM_STATE = config["RANDOM_STATE"]
COMPRESS = config["COMPRESS"]
N_JOBS = config["N_JOBS"]
TARGET = config["TARGET"]

# Add Project Directory to sys.path

In [ ]:
# Add the project directory to sys.path so we can import local modules
sys.path.append(str(PATH))

In [ ]:
from ml_utils import *

# MLflow Tracking URI

In [ ]:
mlflow.set_tracking_uri(f"file://{PATH / 'ml_experiments'}")

# Load Data

In [ ]:
df_train = joblib.load(PATH / 'df_train.pkl')
df_val = joblib.load(PATH / 'df_val.pkl')

# Baseline Models

In [ ]:
# Set MLflow experiment
mlflow_experiment = "baseline_models_2"
mlflow.set_experiment(mlflow_experiment)

# Function to get all regression models
def get_regression_models():
  regressors = []
  # Add scikit-learn regressors
  for name, RegressorClass in all_estimators(type_filter='regressor'):
    try:
      # Exclude models that require additional setup or specific parameters
      if name in [
          'MultiOutputRegressor',
          'StackingRegressor',
          'VotingRegressor',
          'MultiTaskElasticNet',
          'MultiTaskElasticNetCV',
          'MultiTaskLasso',
          'MultiTaskLassoCV'
      ]:
        continue

      regressor = RegressorClass()
      regressors.append((name, regressor))
    except Exception as e:
      print(f"Could not instantiate {name}: {e}")

  # Add XGBoost and LightGBM regressors
  regressors.append(('XGBRegressor', XGBRegressor()))
  regressors.append(('LGBMRegressor', LGBMRegressor()))

  return regressors

# Prepare features and target
X_train = df_train.drop(columns=[TARGET])
y_train = df_train[TARGET]
X_val = df_val.drop(columns=[TARGET])
y_val = df_val[TARGET]

# Log-transform the target variable
# (using log1p to handle zero/negative values safely)
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Get all regression models
models = get_regression_models()

# Dictionary to store results
results_log = {}

# Train and evaluate each model on log-transformed target
for name, model in models:
  try:
    with mlflow.start_run():
      # Train the model on log-transformed target
      model.fit(X_train, y_train_log)

      # Make predictions
      y_pred_log = model.predict(X_val)

      # Transform predictions back to original scale
      y_pred = np.expm1(y_pred_log)

      # Calculate metrics on log scale
      mse_log = mean_squared_error(y_val_log, y_pred_log)
      rmse_log = np.sqrt(mse_log)
      mae_log = mean_absolute_error(y_val_log, y_pred_log)
      r2_log = r2_score(y_val_log, y_pred_log)

      # Calculate metrics on original scale
      mse_orig = mean_squared_error(y_val, y_pred)
      rmse_orig = np.sqrt(mse_orig)
      mae_orig = mean_absolute_error(y_val, y_pred)
      r2_orig = r2_score(y_val, y_pred)

      # Log model and metrics to MLflow
      mlflow.log_param("model_name", name)
      mlflow.log_metric("RMSE_log", rmse_log)
      mlflow.log_metric("MAE_log", mae_log)
      mlflow.log_metric("R2_log", r2_log)
      mlflow.log_metric("RMSE_orig", rmse_orig)
      mlflow.log_metric("MAE_orig", mae_orig)
      mlflow.log_metric("R2_orig", r2_orig)

  except Exception as e:
    print(f"Error with {name} (log-transformed): {e}")

# Modelling

## LinearSVR

In [ ]:
selected_features = [
    'market_id_encoded_scaled',
    'publisher_encoded_scaled',
    'day_of_week_encoded_scaled',
    'category_id_encoded_scaled',
    'market_popularity_transformed_scaled',
    'publisher_avg_clicks_transformed_scaled'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "LinearSVR"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.7, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune LinearSVR parameters
  C = trial.suggest_float('C', 1e-12, 1e-3, log=True)
  epsilon = trial.suggest_float('epsilon', 0.0, 1.0)

  loss = trial.suggest_categorical(
      'loss',
       ['epsilon_insensitive', 'squared_epsilon_insensitive']
  )

  max_iter = trial.suggest_int('max_iter', 500, 5000)
  tol = trial.suggest_float('tol', 1e-5, 1e-2, log=True)
  fit_intercept = True  # Fix to True
  intercept_scaling = trial.suggest_float('intercept_scaling', 1.0, 10.0)
  dual = trial.suggest_categorical('dual', ['auto', True, False])

  # Enforce valid dual setting for loss='epsilon_insensitive'
  if loss == 'epsilon_insensitive' and dual is False:
    dual = True  # Override to a valid value

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx],
        X_train_full.iloc[val_idx]
    )
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Initialize LinearSVR model
    model = LinearSVR(
        C=C,
        epsilon=epsilon,
        loss=loss,
        max_iter=max_iter,
        tol=tol,
        dual=dual,
        fit_intercept=fit_intercept,
        intercept_scaling=intercept_scaling,
        random_state=RANDOM_STATE
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      ),
      load_if_exists=True
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters and trial attributes
  best_params = study.best_params
  best_trial = study.best_trial

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Train final model with best LinearSVR parameters
  filtered_params = {
      k: v
      for k, v in best_params.items()
      if k in LinearSVR().get_params().keys()
  }

  # Ensure dual is valid for the best loss parameter
  if (
      filtered_params['loss'] == 'epsilon_insensitive'
      and filtered_params['dual'] is False
  ):
    filtered_params['dual'] = True

  # Added fit_intercept and random_state here as these are constant values
  final_model = LinearSVR(
      **filtered_params,
      fit_intercept = True,
      random_state=RANDOM_STATE
  )

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)
  mlflow.sklearn.log_model(sk_model=final_model, name="LinearSVR")
  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis using LinearExplainer
  shap_analysis(final_model, X_final, model_type="linear")

## HuberRegressor

In [ ]:
selected_features = [
    'publisher_avg_clicks_transformed_scaled',
    'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled' #,
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks',
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "HuberRegressor"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.9, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune HuberRegressor parameters
  epsilon = trial.suggest_float('epsilon', 1.01, 5.0)
  alpha = trial.suggest_float('alpha', 0.1, 10.0, log=True)
  max_iter = trial.suggest_int('max_iter', 100, 1000)
  tol = trial.suggest_float('tol', 1e-6, 1e-3, log=True)
  fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx],
        X_train_full.iloc[val_idx]
    )
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Initialize HuberRegressor model
    model = HuberRegressor(
        epsilon=epsilon,
        alpha=alpha,
        max_iter=max_iter,
        tol=tol,
        fit_intercept=fit_intercept
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      ),
      load_if_exists=True
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters and trial attributes
  best_params = study.best_params
  best_trial = study.best_trial

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Train final model with best HuberRegressor parameters
  filtered_params = {
      k: v
      for k, v in best_params.items()
      if k in HuberRegressor().get_params().keys()
  }

  # Added fit_intercept as a constant value
  final_model = HuberRegressor(
      **filtered_params,
      fit_intercept = True
  )

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)
  mlflow.sklearn.log_model(sk_model=final_model, name="HuberRegressor")
  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis using LinearExplainer
  shap_analysis(final_model, X_final, model_type="linear")

## BaggingRegressor

In [ ]:
selected_features = [
    'publisher_avg_clicks_transformed_scaled',
    # 'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled',
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks',
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "BaggingRegressor"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.9, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune BaggingRegressor parameters
  n_estimators = trial.suggest_int('n_estimators', 10, 200)
  max_samples = trial.suggest_float('max_samples', 0.1, 1.0)
  max_features = trial.suggest_float('max_features', 0.1, 1.0)
  bootstrap = trial.suggest_categorical('bootstrap', [True, False])

  bootstrap_features = trial.suggest_categorical(
      'bootstrap_features', [True, False]
  )

  oob_score = (
      trial.suggest_categorical('oob_score', [True, False])
      if bootstrap else False
  )

  # Select base estimator
  estimator_name = trial.suggest_categorical('estimator', [
      'DecisionTreeRegressor',
      'LinearRegression',
      'Ridge',
      'KNeighborsRegressor',
      'SVR'
  ])

  # Initialize base estimator with specific hyperparameters
  if estimator_name == 'DecisionTreeRegressor':
    max_depth = trial.suggest_int('max_depth', 3, 15, log=True)
    min_samples_split = trial.suggest_int('min_samples_split', 5, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20)
    criterion = trial.suggest_categorical(
        'criterion',
          ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
    )
    splitter = trial.suggest_categorical('splitter', ['best', 'random'])
    min_weight_fraction_leaf = trial.suggest_float(
        'min_weight_fraction_leaf',
        0.01,
        0.3
    )
    max_features_dt = trial.suggest_float('max_features_dt', 0.1, 1.0)
    max_leaf_nodes = trial.suggest_categorical(
        'max_leaf_nodes',
         [None, 10, 50, 100, 500]
    )
    min_impurity_decrease = trial.suggest_float(
        'min_impurity_decrease',
        0.0,
        0.05
    )
    ccp_alpha = trial.suggest_float('ccp_alpha', 0.0, 0.05)
    base_estimator = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        splitter=splitter,
        min_weight_fraction_leaf=min_weight_fraction_leaf,
        max_features=max_features_dt,
        max_leaf_nodes=max_leaf_nodes,
        min_impurity_decrease=min_impurity_decrease,
        ccp_alpha=ccp_alpha,
        # Set to None as no specific constraints provided
        monotonic_cst=None,
        random_state=RANDOM_STATE
    )
  elif estimator_name == 'LinearRegression':
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    tol = trial.suggest_float('tol', 1e-6, 1e-3, log=True)
    base_estimator = LinearRegression(
        fit_intercept=fit_intercept,
        tol=tol,
        n_jobs=N_JOBS
    )
  elif estimator_name == 'Ridge':
    alpha = trial.suggest_float('alpha', 1e-4, 100.0, log=True)
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    tol = trial.suggest_float('tol', 1e-6, 1e-3, log=True)
    solver = trial.suggest_categorical(
        'solver',
         ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    )
    base_estimator = Ridge(
        alpha=alpha,
        fit_intercept=fit_intercept,
        tol=tol,
        solver=solver,
        random_state=RANDOM_STATE
    )
  elif estimator_name == 'KNeighborsRegressor':
    n_neighbors = trial.suggest_int('n_neighbors', 5, 30)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical(
        'algorithm',
         ['auto', 'ball_tree', 'kd_tree', 'brute']
    )
    leaf_size = trial.suggest_int('leaf_size', 10, 50)
    p = trial.suggest_int('p', 1, 2)
    metric = trial.suggest_categorical(
        'metric',
         ['minkowski', 'euclidean', 'manhattan']
    )
    base_estimator = KNeighborsRegressor(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm,
        leaf_size=leaf_size,
        p=p,
        metric=metric,
        metric_params=None,
        n_jobs=N_JOBS
    )
  elif estimator_name == 'SVR':
    C = trial.suggest_float('C', 1e-2, 5.0, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-2, 0.5, log=True)
    kernel = trial.suggest_categorical(
        'kernel',
         ['rbf', 'linear', 'poly', 'sigmoid']
    )
    degree = trial.suggest_int('degree', 1, 5)
    gamma = trial.suggest_categorical(
        'gamma',
         ['scale', 'auto'] + [1e-3, 1e-2, 1e-1]
    )
    coef0 = trial.suggest_float('coef0', 0.0, 1.0)
    tol = trial.suggest_float('tol', 1e-5, 1e-2, log=True)
    shrinking = trial.suggest_categorical('shrinking', [True, False])
    base_estimator = SVR(
        C=C,
        epsilon=epsilon,
        kernel=kernel,
        degree=degree,
        gamma=gamma,
        coef0=coef0,
        tol=tol,
        shrinking=shrinking
    )

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx],
        X_train_full.iloc[val_idx]
    )
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Initialize BaggingRegressor model
    model = BaggingRegressor(
        estimator=base_estimator,
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        bootstrap=bootstrap,
        bootstrap_features=bootstrap_features,
        oob_score=oob_score,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      ),
      load_if_exists=True
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters and trial attributes
  best_params = study.best_params
  best_trial = study.best_trial

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Create best base estimator
  if best_params['estimator'] == 'DecisionTreeRegressor':
    base_estimator = DecisionTreeRegressor(
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        criterion=best_params['criterion'],
        splitter=best_params['splitter'],
        min_weight_fraction_leaf=best_params['min_weight_fraction_leaf'],
        max_features=best_params['max_features_dt'],
        max_leaf_nodes=best_params['max_leaf_nodes'],
        min_impurity_decrease=best_params['min_impurity_decrease'],
        ccp_alpha=best_params['ccp_alpha'],
        monotonic_cst=None,
        random_state=RANDOM_STATE
    )
  elif best_params['estimator'] == 'LinearRegression':
    base_estimator = LinearRegression(
        fit_intercept=best_params['fit_intercept'],
        tol=best_params['tol'],
        n_jobs=N_JOBS
    )
  elif best_params['estimator'] == 'Ridge':
    base_estimator = Ridge(
        alpha=best_params['alpha'],
        fit_intercept=best_params['fit_intercept'],
        tol=best_params['tol'],
        solver=best_params['solver'],
        random_state=RANDOM_STATE
    )
  elif best_params['estimator'] == 'KNeighborsRegressor':
    base_estimator = KNeighborsRegressor(
        n_neighbors=best_params['n_neighbors'],
        weights=best_params['weights'],
        algorithm=best_params['algorithm'],
        leaf_size=best_params['leaf_size'],
        p=best_params['p'],
        metric=best_params['metric'],
        metric_params=None,
        n_jobs=N_JOBS
    )
  elif best_params['estimator'] == 'SVR':
    base_estimator = SVR(
        C=best_params['C'],
        epsilon=best_params['epsilon'],
        kernel=best_params['kernel'],
        degree=best_params['degree'],
        gamma=best_params['gamma'],
        coef0=best_params['coef0'],
        tol=best_params['tol'],
        shrinking=best_params['shrinking']
    )

  # Train final model with best BaggingRegressor parameters
  filtered_params = {
      k: v
      for k, v in best_params.items()
      if k in BaggingRegressor().get_params().keys() and k != 'estimator'
  }

  final_model = BaggingRegressor(
      estimator=base_estimator,
      **filtered_params,
      random_state=RANDOM_STATE,
      n_jobs=N_JOBS
  )

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)
  mlflow.sklearn.log_model(sk_model=final_model, name="BaggingRegressor")
  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis using KernelExplainer
  shap_analysis(final_model, X_final, model_type="kernel")

## HistGradientBoostingRegressor

In [ ]:
selected_features = [
    # 'publisher_avg_clicks_transformed_scaled',
    # 'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    # 'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled' #,
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks',
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "HistGradientBoostingRegressor"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.85, 0.95)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune HistGradientBoostingRegressor parameters
  learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2, log=True)
  max_iter = trial.suggest_int('max_iter', 100, 1000)
  max_depth = trial.suggest_int('max_depth', 3, 6)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 100, 500)
  l2_regularization = trial.suggest_float('l2_regularization', 1.0, 20.0)
  max_bins = trial.suggest_int('max_bins', 50, 200)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 15, 50)
  tol = trial.suggest_float('tol', 1e-7, 1e-4, log=True)
  validation_fraction = trial.suggest_float('validation_fraction', 0.2, 0.4)
  n_iter_no_change = trial.suggest_int('n_iter_no_change', 5, 100)
  loss = trial.suggest_categorical(
      'loss',
       ['squared_error', 'absolute_error', 'gamma', 'poisson', 'quantile']
  )
  early_stopping = True
  random_state=RANDOM_STATE

  # Tune quantile if loss is quantile
  quantile = None
  if loss == 'quantile':
    quantile = trial.suggest_float('quantile', 0.1, 0.9)

  # Tune interaction constraints
  interaction_cst = trial.suggest_categorical(
      'interaction_cst', ['pairwise', 'no_interactions']
  )

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Noise Level: Add slight Gaussian noise to training data for
    # robustness. The standard deviation of 0.01 is chosen to be small
    # enough to avoid significantly altering the data distribution while
    # introducing enough variation to improve robustness. You can
    # experiment with values like 0.005 or 0.02 if needed, but 0.01 is a
    # common starting point for scaled features.
    # X_train_noisy = X_train + np.random.normal(0, 0.02, X_train.shape)

    # Initialize HistGradientBoostingRegressor model
    model = HistGradientBoostingRegressor(
        learning_rate=learning_rate,
        max_iter=max_iter,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        l2_regularization=l2_regularization,
        max_bins=max_bins,
        max_leaf_nodes=max_leaf_nodes,
        tol=tol,
        validation_fraction=validation_fraction,
        loss=loss,
        random_state=random_state,
        interaction_cst=interaction_cst,
        quantile=quantile if loss == 'quantile' else None,
        n_iter_no_change=n_iter_no_change,
        early_stopping=early_stopping
    )

    model.fit(X_train, y_train)
    # model.fit(X_train_noisy, y_train)
    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model
    # del X_train_noisy, X_val_fold, y_train, y_val_fold, y_pred, model
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      ),
      load_if_exists=True
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters and trial attributes
  best_params = study.best_params
  best_trial = study.best_trial

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Train final model with best HistGradientBoostingRegressor parameters
  filtered_params = {
      k: v
      for k, v in best_params.items()
      if k in HistGradientBoostingRegressor().get_params().keys()
  }

  final_model = HistGradientBoostingRegressor(
      **filtered_params,
      early_stopping=True,
      random_state=RANDOM_STATE
  )

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)

  mlflow.sklearn.log_model(
      sk_model=final_model, name="HistGradientBoostingRegressor"
  )

  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis using TreeExplainer
  shap_analysis(final_model, X_final, model_type="tree")

## NuSVR

In [ ]:
selected_features = [
    'publisher_avg_clicks_transformed_scaled',
    'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled' #,
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks' #,
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "NuSVR"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.9, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune NuSVR parameters
  C = trial.suggest_float('C', 0.001, 5, log=True)  # Expanded range

  kernel = trial.suggest_categorical(
      'kernel', ['linear', 'poly', 'rbf', 'sigmoid']
  )

  nu = trial.suggest_float('nu', 0.05, 1.0)  # Lowered minimum
  shrinking = trial.suggest_categorical('shrinking', [True, False])
  tol = trial.suggest_float('tol', 1e-6, 1e-1, log=True)

  # Parameters specific to certain kernels
  degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

  coef0 = (
      trial.suggest_float('coef0', 0.0, 1.0)
      if kernel in ['poly', 'sigmoid'] else 0.0
  )

  if kernel in ['rbf', 'poly', 'sigmoid']:
    gamma_choice = trial.suggest_categorical(
        'gamma_choice',
         ['scale', 'auto', 'float']
    )
    if gamma_choice == 'float':
      gamma = trial.suggest_float('gamma_float', 1e-4, 1e-1, log=True)
    else:
      gamma = gamma_choice
  else:
    gamma = 'scale'

  # Tune Gaussian noise standard deviation
  noise_std = trial.suggest_float('noise_std', 0.01, 0.02, log=True)

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx],
        X_train_full.iloc[val_idx]
    )
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Add Gaussian noise to training data for robustness
    X_train_noisy = X_train + np.random.normal(0, noise_std, X_train.shape)

    # Initialize NuSVR model
    model = NuSVR(
        C=C,
        kernel=kernel,
        nu=nu,
        # shrinking=shrinking,
        tol=tol,
        degree=degree,
        coef0=coef0,
        gamma=gamma
    )

    # model.fit(X_train, y_train)
    model.fit(X_train_noisy, y_train)
    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    # del X_train, X_val_fold, y_train, y_val_fold, y_pred, model
    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model, X_train_noisy
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      ),
      load_if_exists=True
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters
  best_params = study.best_params

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Train final model with best NuSVR parameters
  filtered_params = {
      k: v
      for k, v in best_params.items()
      if k in NuSVR().get_params().keys()
  }

  final_model = NuSVR(**filtered_params)

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)

  mlflow.sklearn.log_model(sk_model=final_model, name="NuSVR")

  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis using KernelExplainer
  shap_analysis(final_model, X_final, model_type="kernel")

## XGBoostRegressor

In [ ]:
selected_features = [
    'publisher_avg_clicks_transformed_scaled',
    # 'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled' #,
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks',
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "XGBoostRegressor"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.9, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune XGBoost parameters
  tree_method = trial.suggest_categorical(
      'tree_method', ['auto', 'exact', 'approx', 'hist']
  )

  params = {
      'base_score': trial.suggest_float('base_score', 0.3, 0.7),
      'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
      'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 0.9),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
      'early_stopping_rounds': trial.suggest_int(
          'early_stopping_rounds', 20, 60
      ),
      'gamma': trial.suggest_float('gamma', 0.1, 3),
      'grow_policy': trial.suggest_categorical(
          'grow_policy', ['depthwise', 'lossguide']
      ),
      'importance_type': trial.suggest_categorical(
          'importance_type', ['gain', 'weight', 'cover']
      ),
      'learning_rate': trial.suggest_float(
          'learning_rate', 0.01, 0.2, log=True
      ),
      'max_bin': trial.suggest_int('max_bin', 256, 768, step=128),
      'max_delta_step': trial.suggest_float('max_delta_step', 0, 5),
      'max_depth': trial.suggest_int('max_depth', 3, 8),
      'max_leaves': trial.suggest_int('max_leaves', 0, 32),
      'min_child_weight': trial.suggest_float('min_child_weight', 1, 8),
      'n_estimators': trial.suggest_int('n_estimators', 100, 800),
      'num_parallel_tree': trial.suggest_int('num_parallel_tree', 1, 5),
      'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 5),
      'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 5),
      'sampling_method': trial.suggest_categorical(
          'sampling_method', ['uniform', 'gradient_based']
      ),
      'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.5, 2.0),
      'subsample': trial.suggest_float('subsample', 0.7, 0.95),
      'tree_method': tree_method,
      'objective': 'reg:squarederror',
      'eval_metric': 'rmse',
      'n_jobs': N_JOBS,
      'random_state': RANDOM_STATE,
      'validate_parameters': True
  }

  # Only include colsample_bynode if tree_method is not 'exact'
  if tree_method != 'exact':
    params['colsample_bynode'] = trial.suggest_float(
        'colsample_bynode', 0.5, 1.0
    )

  # Tune Gaussian noise standard deviation
  noise_std = trial.suggest_float('noise_std', 0.001, 0.015, log=True)

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx],
        X_train_full.iloc[val_idx]
    )
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Add Gaussian noise to training data
    X_train_noisy = X_train + np.random.normal(0, noise_std, X_train.shape)

    # Initialize XGBRegressor model
    model = XGBRegressor(**params)

    # Fit with early stopping
    model.fit(
        X_train_noisy, y_train,
        # X_train, y_train,
        eval_set=[(X_val_fold, y_val_fold)],
        verbose=False
    )

    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model, X_train_noisy
    # del X_train, X_val_fold, y_train, y_val_fold, y_pred, model
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      ),
      load_if_exists=True
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters
  best_params = study.best_params

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Train final model with best XGBoost parameters
  filtered_params = {
      k: v
      for k, v in best_params.items()
      if k in XGBRegressor().get_params().keys()
  }

  final_model = XGBRegressor(
      **filtered_params,
      objective='reg:squarederror',
      eval_metric='rmse',
      n_jobs=N_JOBS,
      random_state=RANDOM_STATE,
      sampling_method='uniform',
      validate_parameters=True
  )

  X_final = X_final[sorted(X_final.columns)]

  final_model.fit(
      X_final, y_train_full,
      eval_set=[(X_final, y_train_full)],
      verbose=False
  )

  mlflow.xgboost.log_model(xgb_model=final_model, name="XGBoostRegressor")

  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis using TreeExplainer (more efficient for tree-based models)
  shap_analysis(final_model, X_final, model_type="tree")

## LGBMRegressor

In [ ]:
selected_features = [
    'publisher_avg_clicks_transformed_scaled',
    'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled',
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks',
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "LightGBMRegressor"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.95, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune LightGBM parameters
  boosting_type = trial.suggest_categorical(
      'boosting_type',
       ['gbdt', 'dart', 'goss']
  )

  params = {
      'boosting_type': boosting_type,
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
      'importance_type': trial.suggest_categorical(
          'importance_type',
           ['split', 'gain']
      ),
      'learning_rate': trial.suggest_float(
          'learning_rate',
          0.005,
          0.1,
          log=True
      ),
      'max_depth': trial.suggest_int('max_depth', 3, 6),
      'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
      'min_child_weight': trial.suggest_float(
          'min_child_weight',
          1e-3,
          5,
          log=True
      ),
      'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.2),
      'n_estimators': trial.suggest_int('n_estimators', 50, 500),
      'n_jobs': N_JOBS,
      'num_leaves': trial.suggest_int('num_leaves', 10, 50),
      'objective': 'regression',
      'random_state': RANDOM_STATE,
      'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0),
      'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0),
      'subsample': trial.suggest_float('subsample', 0.6, 0.9),
      'subsample_for_bin': trial.suggest_int(
          'subsample_for_bin',
          50000,
          150000,
          step=50000
      ),
      'subsample_freq': trial.suggest_int('subsample_freq', 1, 5),
      'verbose': -1  # Suppress log
  }

  # Tune Gaussian noise standard deviation
  noise_std = trial.suggest_float('noise_std', 0.001, 0.02, log=True)

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
    )

    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Add Gaussian noise to training data
    X_train_noisy = X_train + np.random.normal(0, noise_std, X_train.shape)

    # Initialize LightGBM model
    model = LGBMRegressor(**params)

    # Fit model
    model.fit(
        X_train_noisy, y_train,
        eval_set=[(X_val_fold, y_val_fold)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
    )

    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model, X_train_noisy
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      ),
      load_if_exists=True
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances: {fanova_importances}')

  # Get best parameters
  best_params = study.best_params

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Train final model with best LightGBM parameters
  filtered_params = {
      k: v for k, v in best_params.items()
      if k in LGBMRegressor().get_params().keys()
  }

  final_model = LGBMRegressor(
      **filtered_params,
      objective='regression',
      metric='rmse',
      n_jobs=N_JOBS,
      random_state=RANDOM_STATE,
      verbose=-1
  )

  X_final = X_final[sorted(X_final.columns)]

  final_model.fit(
      X_final, y_train_full,
      eval_set=[(X_final, y_train_full)],
      eval_metric='rmse',
      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
  )

  mlflow.lightgbm.log_model(lgb_model=final_model, name="LightGBMRegressor")

  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis using TreeExplainer
  shap_analysis(final_model, X_final, model_type="tree")

## VotingRegressor

In [ ]:
# Initialize estimators with type-converted parameters
best_params_linear_svr = get_best_params("LinearSVR")

# Ensure valid parameter combination for LinearSVR
loss = best_params_linear_svr['loss']
dual = convert_param(best_params_linear_svr['dual'], 'bool', 'dual')

if loss == 'epsilon_insensitive' and not dual:
  dual = True  # Force dual=True for epsilon_insensitive to avoid ValueError

linear_svr = LinearSVR(
    C=convert_param(best_params_linear_svr['C'], 'float', 'C'),
    epsilon=convert_param(
        best_params_linear_svr['epsilon'], 'float', 'epsilon'
    ),
    loss=loss,
    max_iter=convert_param(
        best_params_linear_svr['max_iter'], 'int', 'max_iter'
    ),
    tol=convert_param(best_params_linear_svr['tol'], 'float', 'tol'),
    dual=dual,
    intercept_scaling=convert_param(
        best_params_linear_svr['intercept_scaling'],
        'float',
        'intercept_scaling'
    ),
    random_state=RANDOM_STATE
)

best_params_huber_regressor = get_best_params("HuberRegressor")

huber_regressor = HuberRegressor(
    epsilon=convert_param(
        best_params_huber_regressor['epsilon'],
        'float',
        'epsilon'
    ),
    alpha=convert_param(best_params_huber_regressor['alpha'], 'float', 'alpha'),
    max_iter=convert_param(
        best_params_huber_regressor['max_iter'],
        'int',
        'max_iter'
    ),
    tol=convert_param(best_params_huber_regressor['tol'], 'float', 'tol')
)

best_params_bagging_regressor = get_best_params("BaggingRegressor")

bagging_regressor = BaggingRegressor(
    estimator=LinearRegression(),
    n_estimators=convert_param(
        best_params_bagging_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    max_samples=convert_param(
        best_params_bagging_regressor['max_samples'],
        'float',
        'max_samples'
    ),
    max_features=convert_param(
        best_params_bagging_regressor['max_features'],
        'float',
        'max_features'
    ),
    bootstrap=convert_param(
        best_params_bagging_regressor['bootstrap'],
        'bool',
        'bootstrap'
    ),
    bootstrap_features=convert_param(
        best_params_bagging_regressor['bootstrap_features'],
        'bool',
        'bootstrap_features'
    ),
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS
)

best_params_hist_gradient_boosting_regressor = (
    get_best_params("HistGradientBoostingRegressor")
)

hist_gradient_boosting_regressor = HistGradientBoostingRegressor(
    learning_rate=convert_param(
        best_params_hist_gradient_boosting_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_iter=convert_param(
        best_params_hist_gradient_boosting_regressor['max_iter'],
        'int',
        'max_iter'
    ),
    max_depth=convert_param(
        best_params_hist_gradient_boosting_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    min_samples_leaf=convert_param(
        best_params_hist_gradient_boosting_regressor['min_samples_leaf'],
        'int',
        'min_samples_leaf'
    ),
    l2_regularization=convert_param(
        best_params_hist_gradient_boosting_regressor['l2_regularization'],
        'float',
        'l2_regularization'
    ),
    max_bins=convert_param(
        best_params_hist_gradient_boosting_regressor['max_bins'],
        'int',
        'max_bins'
    ),
    max_leaf_nodes=convert_param(
        best_params_hist_gradient_boosting_regressor['max_leaf_nodes'],
        'int',
        'max_leaf_nodes'
    ),
    tol=convert_param(
        best_params_hist_gradient_boosting_regressor['tol'], 'float', 'tol'
    ),
    validation_fraction=convert_param(
        best_params_hist_gradient_boosting_regressor['validation_fraction'],
        'float',
        'validation_fraction'
    ),
    loss=best_params_hist_gradient_boosting_regressor['loss'],  # Categorical
    random_state=RANDOM_STATE,
    interaction_cst=(
        best_params_hist_gradient_boosting_regressor['interaction_cst']
    ),
    n_iter_no_change=convert_param(
        best_params_hist_gradient_boosting_regressor['n_iter_no_change'],
        'int',
        'n_iter_no_change'
    )
)

best_params_nu_svr = get_best_params("NuSVR")

nu_svr = NuSVR(
    C=convert_param(best_params_nu_svr['C'], 'float', 'C'),
    kernel=best_params_nu_svr['kernel'],
    nu=convert_param(best_params_nu_svr['nu'], 'float', 'nu'),
    shrinking=convert_param(
        best_params_nu_svr['shrinking'],'bool',
        'shrinking'
    ),
    tol=convert_param(best_params_nu_svr['tol'], 'float', 'tol')
)

best_params_xgb_regressor = get_best_params("XGBoostRegressor")

# Remove early_stopping_rounds from best_params_xgb_regressor to avoid conflicts
best_params_xgb_regressor.pop('early_stopping_rounds', None)

xgb_regressor = XGBRegressor(
    base_score=convert_param(
        best_params_xgb_regressor['base_score'],
        'float',
        'base_score'
    ),
    booster=best_params_xgb_regressor['booster'],
    colsample_bylevel=convert_param(
        best_params_xgb_regressor['colsample_bylevel'],
        'float',
        'colsample_bylevel'
    ),
    colsample_bytree=convert_param(
        best_params_xgb_regressor['colsample_bytree'],
        'float',
        'colsample_bytree'
    ),
    gamma=convert_param(best_params_xgb_regressor['gamma'], 'float', 'gamma'),
    grow_policy=best_params_xgb_regressor['grow_policy'],
    importance_type=best_params_xgb_regressor['importance_type'],
    learning_rate=convert_param(
        best_params_xgb_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_bin=convert_param(
        best_params_xgb_regressor['max_bin'],
        'int',
        'max_bin'
    ),
    max_delta_step=convert_param(
        best_params_xgb_regressor['max_delta_step'],
        'float',
        'max_delta_step'
    ),
    max_depth=convert_param(
        best_params_xgb_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    max_leaves=convert_param(
        best_params_xgb_regressor['max_leaves'],
        'int',
        'max_leaves'
    ),
    min_child_weight=convert_param(
        best_params_xgb_regressor['min_child_weight'],
        'float',
        'min_child_weight'
    ),
    n_estimators=convert_param(
        best_params_xgb_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    num_parallel_tree=convert_param(
        best_params_xgb_regressor['num_parallel_tree'],
        'int',
        'num_parallel_tree'
    ),
    reg_alpha=convert_param(
        best_params_xgb_regressor['reg_alpha'],
        'float',
        'reg_alpha'
    ),
    reg_lambda=convert_param(
        best_params_xgb_regressor['reg_lambda'],
        'float',
        'reg_lambda'
    ),
    subsample=convert_param(
        best_params_xgb_regressor['subsample'],
        'float',
        'subsample'
    ),
    tree_method=best_params_xgb_regressor['tree_method'],
    objective='reg:squarederror',
    eval_metric='rmse',
    n_jobs=N_JOBS,
    random_state=RANDOM_STATE,
    sampling_method='uniform'
)

best_params_lgbm_regressor = get_best_params("LightGBMRegressor")

lgbm_regressor = LGBMRegressor(
    boosting_type=best_params_lgbm_regressor['boosting_type'],
    colsample_bytree=convert_param(
        best_params_lgbm_regressor['colsample_bytree'],
        'float',
        'colsample_bytree'
    ),
    importance_type=best_params_lgbm_regressor['importance_type'],
    learning_rate=convert_param(
        best_params_lgbm_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_depth=convert_param(
        best_params_lgbm_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    min_child_samples=convert_param(
        best_params_lgbm_regressor['min_child_samples'],
        'int',
        'min_child_samples'
    ),
    min_child_weight=convert_param(
        best_params_lgbm_regressor['min_child_weight'],
        'float',
        'min_child_weight'
    ),
    min_split_gain=convert_param(
        best_params_lgbm_regressor['min_split_gain'],
        'float',
        'min_split_gain'
    ),
    n_estimators=convert_param(
        best_params_lgbm_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    num_leaves=convert_param(
        best_params_lgbm_regressor['num_leaves'],
        'int',
        'num_leaves'
    ),
    reg_alpha=convert_param(
        best_params_lgbm_regressor['reg_alpha'],
        'float',
        'reg_alpha'
    ),
    reg_lambda=convert_param(
        best_params_lgbm_regressor['reg_lambda'],
        'float',
        'reg_lambda'
    ),
    subsample=convert_param(
        best_params_lgbm_regressor['subsample'],
        'float',
        'subsample'
    ),
    subsample_for_bin=convert_param(
        best_params_lgbm_regressor['subsample_for_bin'],
        'int',
        'subsample_for_bin'
    ),
    subsample_freq=convert_param(
        best_params_lgbm_regressor['subsample_freq'],
        'int',
        'subsample_freq'
    ),
    n_jobs=N_JOBS,
    random_state=RANDOM_STATE,
    verbose=-1,
    objective='regression',
    metric='rmse'
)

selected_features = [
    'publisher_avg_clicks_transformed_scaled',
    'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled',
    'is_weekend_encoded_scaled',
    'pub_popularity_x_pub_clicks',
    'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "VotingRegressor"
study_name = f"{mlflow_experiment}_study"
storage = f"sqlite:///{PATH}/{study_name}.db"

# Define base estimators
estimators = [
    ('linear_svr', linear_svr),
    ('huber_regressor', huber_regressor),
    ('bagging_regressor', bagging_regressor),
    ('hist_gradient_boosting', hist_gradient_boosting_regressor),
    ('nu_svr', nu_svr),
    ('xgb_regressor', xgb_regressor),
    ('lgbm_regressor', lgbm_regressor)
]

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.9, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune weights for VotingRegressor using estimator names
  weights = []

  for name, _ in estimators:
    weights.append(trial.suggest_float(f'weight_{name}', 0.0, 1.0))

  weights = np.array(weights)
  weights = weights / weights.sum()  # Normalize weights to sum to 1

  # Tune Gaussian noise standard deviation
  noise_std = trial.suggest_float('noise_std', 0.001, 0.02, log=True)

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
    )

    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Add Gaussian noise to training data and preserve feature names
    noise = np.random.normal(0, noise_std, X_train.shape)

    X_train_noisy = pd.DataFrame(
        X_train + noise, columns=X_train.columns, index=X_train.index
    )

    # Initialize VotingRegressor with estimators
    model = VotingRegressor(
        estimators=estimators, weights=weights, n_jobs=N_JOBS
    )

    # Fit VotingRegressor
    model.fit(X_train_noisy, y_train)

    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del (
        X_train,
        X_val_fold,
        y_train,
        y_val_fold,
        y_pred, model,
        X_train_noisy,
        noise
    )

    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      load_if_exists=True,
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      )
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters
  best_params = study.best_params

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Prepare weights for final model using estimator names
  weights = np.array(
      [best_params.get(f'weight_{name}', 0.0) for name, _ in estimators]
  )
  weights = weights / weights.sum()  # Normalize weights

  # Train final VotingRegressor
  final_model = VotingRegressor(
      estimators=estimators, weights=weights, n_jobs=N_JOBS
  )

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)

  mlflow.sklearn.log_model(final_model, name="VotingRegressor")

  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis
  shap_analysis(final_model, X_final, model_type="kernel")

## StackingRegressor - Baseline

In [ ]:
# Define excluded estimators globally
excluded_estimators = [
    'MultiOutputRegressor',
    'StackingRegressor',
    'VotingRegressor',
    'MultiTaskElasticNet',
    'MultiTaskElasticNetCV',
    'MultiTaskLasso',
    'MultiTaskLassoCV',
    'GammaRegressor',
    'PoissonRegressor',
    'TweedieRegressor',
    'PLSRegression',
    'RegressorChain'
]

# Function to get all regression models
def get_regression_models():
  regressors = []

  # Add scikit-learn regressors
  for name, RegressorClass in all_estimators(type_filter='regressor'):
    try:
      # Exclude models that require additional setup or specific parameters
      if name in excluded_estimators:
        continue

      regressor = RegressorClass()
      regressors.append((name, regressor))
    except Exception as e:
      print(f"Could not instantiate {name}: {e}")

  # Add XGBoost and LightGBM regressors
  regressors.append(('XGBRegressor', XGBRegressor()))
  regressors.append(('LGBMRegressor', LGBMRegressor()))
  return regressors

# Initialize estimators with type-converted parameters
best_params_linear_svr = get_best_params("LinearSVR")
loss = best_params_linear_svr['loss']
dual = convert_param(best_params_linear_svr['dual'], 'bool', 'dual')

if loss == 'epsilon_insensitive' and not dual:
  dual = True  # Force dual=True for epsilon_insensitive to avoid ValueError

linear_svr = LinearSVR(
    C=convert_param(best_params_linear_svr['C'], 'float', 'C'),
    epsilon=convert_param(
        best_params_linear_svr['epsilon'],
        'float',
        'epsilon'
    ),
    loss=loss,
    max_iter=convert_param(
        best_params_linear_svr['max_iter'],
        'int',
        'max_iter'
    ),
    tol=convert_param(best_params_linear_svr['tol'], 'float', 'tol'),
    dual=dual,
    intercept_scaling=convert_param(
        best_params_linear_svr['intercept_scaling'],
        'float',
        'intercept_scaling'
    ),
    random_state=RANDOM_STATE
)

best_params_huber_regressor = get_best_params("HuberRegressor")

huber_regressor = HuberRegressor(
    epsilon=convert_param(
        best_params_huber_regressor['epsilon'],
        'float',
        'epsilon'
    ),
    alpha=convert_param(best_params_huber_regressor['alpha'], 'float', 'alpha'),
    max_iter=convert_param(
        best_params_huber_regressor['max_iter'],
        'int',
        'max_iter'
    ),
    tol=convert_param(best_params_huber_regressor['tol'], 'float', 'tol')
)

best_params_bagging_regressor = get_best_params("BaggingRegressor")

bagging_regressor = BaggingRegressor(
    estimator=HuberRegressor(),
    n_estimators=convert_param(
        best_params_bagging_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    max_samples=convert_param(
        best_params_bagging_regressor['max_samples'],
        'float',
        'max_samples'
    ),
    max_features=convert_param(
        best_params_bagging_regressor['max_features'],
        'float',
        'max_features'
    ),
    bootstrap=convert_param(
        best_params_bagging_regressor['bootstrap'],
        'bool',
        'bootstrap'
    ),
    bootstrap_features=convert_param(
        best_params_bagging_regressor['bootstrap_features'],
        'bool',
        'bootstrap_features'
    ),
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS
)

best_params_hist_gradient_boosting_regressor = get_best_params(
    "HistGradientBoostingRegressor"
)

hist_gradient_boosting_regressor = HistGradientBoostingRegressor(
    learning_rate=convert_param(
        best_params_hist_gradient_boosting_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_iter=convert_param(
        best_params_hist_gradient_boosting_regressor['max_iter'],
        'int',
        'max_iter'
    ),
    max_depth=convert_param(
        best_params_hist_gradient_boosting_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    min_samples_leaf=convert_param(
        best_params_hist_gradient_boosting_regressor['min_samples_leaf'],
        'int',
        'min_samples_leaf'
    ),
    l2_regularization=convert_param(
        best_params_hist_gradient_boosting_regressor['l2_regularization'],
        'float',
        'l2_regularization'
    ),
    max_bins=convert_param(
        best_params_hist_gradient_boosting_regressor['max_bins'],
        'int',
        'max_bins'
    ),
    max_leaf_nodes=convert_param(
        best_params_hist_gradient_boosting_regressor['max_leaf_nodes'],
        'int',
        'max_leaf_nodes'
    ),
    tol=convert_param(
        best_params_hist_gradient_boosting_regressor['tol'], 'float', 'tol'
    ),
    validation_fraction=convert_param(
        best_params_hist_gradient_boosting_regressor['validation_fraction'],
        'float',
        'validation_fraction'
    ),
    loss=best_params_hist_gradient_boosting_regressor['loss'],
    random_state=RANDOM_STATE,
    interaction_cst=best_params_hist_gradient_boosting_regressor[
        'interaction_cst'
    ],
    n_iter_no_change=convert_param(
        best_params_hist_gradient_boosting_regressor['n_iter_no_change'],
        'int',
        'n_iter_no_change'
    )
)

best_params_nu_svr = get_best_params("NuSVR")

nu_svr = NuSVR(
    C=convert_param(best_params_nu_svr['C'], 'float', 'C'),
    kernel=best_params_nu_svr['kernel'],
    nu=convert_param(best_params_nu_svr['nu'], 'float', 'nu'),
    shrinking=convert_param(
        best_params_nu_svr['shrinking'], 'bool', 'shrinking'
    ),
    tol=convert_param(best_params_nu_svr['tol'], 'float', 'tol')
)

best_params_xgb_regressor = get_best_params("XGBoostRegressor")
best_params_xgb_regressor.pop('early_stopping_rounds', None)

xgb_regressor = XGBRegressor(
    base_score=convert_param(
        best_params_xgb_regressor['base_score'],
        'float',
        'base_score'
    ),
    booster=best_params_xgb_regressor['booster'],
    colsample_bylevel=convert_param(
        best_params_xgb_regressor['colsample_bylevel'],
        'float',
        'colsample_bylevel'
    ),
    colsample_bytree=convert_param(
        best_params_xgb_regressor['colsample_bytree'],
        'float',
        'colsample_bytree'
    ),
    gamma=convert_param(best_params_xgb_regressor['gamma'], 'float', 'gamma'),
    grow_policy=best_params_xgb_regressor['grow_policy'],
    importance_type=best_params_xgb_regressor['importance_type'],
    learning_rate=convert_param(
        best_params_xgb_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_bin=convert_param(
        best_params_xgb_regressor['max_bin'],
        'int',
        'max_bin'
    ),
    max_delta_step=convert_param(
        best_params_xgb_regressor['max_delta_step'],
        'float',
        'max_delta_step'
    ),
    max_depth=convert_param(
        best_params_xgb_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    max_leaves=convert_param(
        best_params_xgb_regressor['max_leaves'],
        'int',
        'max_leaves'
    ),
    min_child_weight=convert_param(
        best_params_xgb_regressor['min_child_weight'],
        'float',
        'min_child_weight'
    ),
    n_estimators=convert_param(
        best_params_xgb_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    num_parallel_tree=convert_param(
        best_params_xgb_regressor['num_parallel_tree'],
        'int',
        'num_parallel_tree'
    ),
    reg_alpha=convert_param(
        best_params_xgb_regressor['reg_alpha'],
        'float',
        'reg_alpha'
    ),
    reg_lambda=convert_param(
        best_params_xgb_regressor['reg_lambda'],
        'float',
        'reg_lambda'
    ),
    subsample=convert_param(
        best_params_xgb_regressor['subsample'],
        'float',
        'subsample'
    ),
    tree_method=best_params_xgb_regressor['tree_method'],
    objective='reg:squarederror',
    eval_metric='rmse',
    n_jobs=N_JOBS,
    random_state=RANDOM_STATE,
    sampling_method='uniform'
)

best_params_lgbm_regressor = get_best_params("LightGBMRegressor")

lgbm_regressor = LGBMRegressor(
    boosting_type=best_params_lgbm_regressor['boosting_type'],
    colsample_bytree=convert_param(
        best_params_lgbm_regressor['colsample_bytree'],
        'float',
        'colsample_bytree'
    ),
    importance_type=best_params_lgbm_regressor['importance_type'],
    learning_rate=convert_param(
        best_params_lgbm_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_depth=convert_param(
        best_params_lgbm_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    min_child_samples=convert_param(
        best_params_lgbm_regressor['min_child_samples'],
        'int',
        'min_child_samples'
    ),
    min_child_weight=convert_param(
        best_params_lgbm_regressor['min_child_weight'],
        'float',
        'min_child_weight'
    ),
    min_split_gain=convert_param(
        best_params_lgbm_regressor['min_split_gain'],
        'float',
        'min_split_gain'
    ),
    n_estimators=convert_param(
        best_params_lgbm_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    num_leaves=convert_param(
        best_params_lgbm_regressor['num_leaves'],
        'int',
        'num_leaves'
    ),
    reg_alpha=convert_param(
        best_params_lgbm_regressor['reg_alpha'],
        'float',
        'reg_alpha'
    ),
    reg_lambda=convert_param(
        best_params_lgbm_regressor['reg_lambda'],
        'float',
        'reg_lambda'
    ),
    subsample=convert_param(
        best_params_lgbm_regressor['subsample'],
        'float',
        'subsample'
    ),
    subsample_for_bin=convert_param(
        best_params_lgbm_regressor['subsample_for_bin'],
        'int',
        'subsample_for_bin'
    ),
    subsample_freq=convert_param(
        best_params_lgbm_regressor['subsample_freq'],
        'int',
        'subsample_freq'
    ),
    n_jobs=N_JOBS,
    random_state=RANDOM_STATE,
    verbose=-1,
    objective='regression',
    metric='rmse'
)

selected_features = [
    'publisher_avg_clicks_transformed_scaled',
    'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    # 'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled' #,
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks',
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "StackingRegressor"
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
study_name = f"{mlflow_experiment}_{current_time}"
storage = f"sqlite:///{PATH}/{study_name}.db"

# Define base estimators
estimators = [
    ('linear_svr', linear_svr),
    ('huber_regressor', huber_regressor),
    ('bagging_regressor', bagging_regressor),
    ('hist_gradient_boosting', hist_gradient_boosting_regressor),
    ('nu_svr', nu_svr),
    ('xgb_regressor', xgb_regressor),
    ('lgbm_regressor', lgbm_regressor)
]

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.9, 1)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune whether to add noise
  add_noise = trial.suggest_categorical('add_noise', [True, False])
  noise_std = 0.0

  if add_noise:
    noise_std = trial.suggest_float('noise_std', 0.001, 0.02, log=True)

  # Select final estimator from regression models
  regressor_names = [name for name, _ in get_regression_models()]

  final_estimator_name = trial.suggest_categorical(
      'final_estimator',
      regressor_names
  )

  # Explicitly exclude problematic estimators
  if final_estimator_name in excluded_estimators:
    raise optuna.TrialPruned(
        f"Final estimator {final_estimator_name} is excluded."
    )

  # Initialize final estimator with default parameters
  if final_estimator_name == 'LinearRegression':
    final_estimator = LinearRegression()
  elif final_estimator_name == 'Ridge':
    final_estimator = Ridge(random_state=RANDOM_STATE)
  elif final_estimator_name == 'Lasso':
    final_estimator = Lasso(random_state=RANDOM_STATE)
  elif final_estimator_name == 'ElasticNet':
    final_estimator = ElasticNet(random_state=RANDOM_STATE)
  elif final_estimator_name == 'SVR':
    final_estimator = SVR()
  elif final_estimator_name == 'LinearSVR':
    final_estimator = LinearSVR(random_state=RANDOM_STATE)
  elif final_estimator_name == 'HuberRegressor':
    final_estimator = HuberRegressor()
  elif final_estimator_name == 'RandomForestRegressor':
    final_estimator = RandomForestRegressor(
        random_state=RANDOM_STATE, n_jobs=N_JOBS
    )
  elif final_estimator_name == 'GradientBoostingRegressor':
    final_estimator = GradientBoostingRegressor(random_state=RANDOM_STATE)
  elif final_estimator_name == 'XGBRegressor':
    final_estimator = XGBRegressor(
        objective='reg:squarederror', n_jobs=N_JOBS, random_state=RANDOM_STATE
    )
  elif final_estimator_name == 'LGBMRegressor':
    final_estimator = LGBMRegressor(
        objective='regression',
        n_jobs=N_JOBS,
        random_state=RANDOM_STATE,
        verbose=-1
    )
  else:
    # For other regressors, use default parameters
    for name, regressor in get_regression_models():
      if name == final_estimator_name:
        final_estimator = regressor
        break

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx],
        X_train_full.iloc[val_idx]
    )
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Add Gaussian noise to training data if selected
    if add_noise:
      noise = np.random.normal(0, noise_std, X_train.shape)

      X_train_noisy = pd.DataFrame(
          X_train + noise, columns=X_train.columns, index=X_train.index
      )
    else:
      X_train_noisy = X_train

    # Initialize StackingRegressor
    model = StackingRegressor(
        estimators=estimators,
        final_estimator=final_estimator,
        n_jobs=N_JOBS,
        cv=5,
        passthrough=False
    )

    # Fit StackingRegressor
    try:
      model.fit(X_train_noisy, y_train)
    except ValueError as e:
      print(f"Error fitting model with {final_estimator_name}: {e}")
      raise optuna.TrialPruned(f"Failed to fit {final_estimator_name}: {e}")

    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model, X_train_noisy

    if add_noise:
      del noise
    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      load_if_exists=True,
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      )
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target=(
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters
  best_params = study.best_params

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Prepare final estimator for final model with default parameters
  final_estimator_name = best_params['final_estimator']

  if final_estimator_name in excluded_estimators:
    raise ValueError(
        f"Best final estimator {final_estimator_name} is excluded."
    )
  if final_estimator_name == 'LinearRegression':
    final_estimator = LinearRegression()
  elif final_estimator_name == 'Ridge':
    final_estimator = Ridge(random_state=RANDOM_STATE)
  elif final_estimator_name == 'Lasso':
    final_estimator = Lasso(random_state=RANDOM_STATE)
  elif final_estimator_name == 'ElasticNet':
    final_estimator = ElasticNet(random_state=RANDOM_STATE)
  elif final_estimator_name == 'SVR':
    final_estimator = SVR()
  elif final_estimator_name == 'LinearSVR':
    final_estimator = LinearSVR(random_state=RANDOM_STATE)
  elif final_estimator_name == 'HuberRegressor':
    final_estimator = HuberRegressor()
  elif final_estimator_name == 'RandomForestRegressor':
    final_estimator = RandomForestRegressor(
        random_state=RANDOM_STATE, n_jobs=N_JOBS
    )
  elif final_estimator_name == 'GradientBoostingRegressor':
    final_estimator = GradientBoostingRegressor(random_state=RANDOM_STATE)
  elif final_estimator_name == 'XGBRegressor':
    final_estimator = XGBRegressor(
        objective='reg:squarederror', n_jobs=N_JOBS, random_state=RANDOM_STATE
    )
  elif final_estimator_name == 'LGBMRegressor':
    final_estimator = LGBMRegressor(
        objective='regression',
        n_jobs=N_JOBS,
        random_state=RANDOM_STATE,
        verbose=-1
    )
  else:
    # For other regressors, use default parameters
    for name, regressor in get_regression_models():
      if name == final_estimator_name:
        final_estimator = regressor
        break

  # Train final StackingRegressor
  final_model = StackingRegressor(
      estimators=estimators,
      final_estimator=final_estimator,
      n_jobs=N_JOBS,
      cv=5,
      passthrough=False
  )

  # Add noise to final training data if selected
  if best_params['add_noise']:
    noise = np.random.normal(0, best_params['noise_std'], X_final.shape)

    X_final = pd.DataFrame(
        X_final + noise, columns=X_final.columns, index=X_final.index
    )

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)

  mlflow.sklearn.log_model(final_model, name="StackingRegressor")

  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  evaluate_and_log_metrics(y_val, y_pred, prefix="val")

  # SHAP analysis
  shap_analysis(final_model, X_final, model_type="kernel")

## StackingRegressor - LassoLarsCV

In [ ]:
# Initialize base estimators with type-converted parameters
best_params_linear_svr = get_best_params("LinearSVR")
loss = best_params_linear_svr['loss']
dual = convert_param(best_params_linear_svr['dual'], 'bool', 'dual')

if loss == 'epsilon_insensitive' and not dual:
  dual = True  # Force dual=True for epsilon_insensitive to avoid ValueError

linear_svr = LinearSVR(
    C=convert_param(best_params_linear_svr['C'], 'float', 'C'),
    epsilon=convert_param(
        best_params_linear_svr['epsilon'],
        'float',
        'epsilon'
    ),
    loss=loss,
    max_iter=convert_param(
        best_params_linear_svr['max_iter'],
        'int',
        'max_iter'
    ),
    tol=convert_param(best_params_linear_svr['tol'], 'float', 'tol'),
    dual=dual,
    intercept_scaling=convert_param(
        best_params_linear_svr['intercept_scaling'],
        'float',
        'intercept_scaling'
    ),
    random_state=RANDOM_STATE
)

best_params_huber_regressor = get_best_params("HuberRegressor")

huber_regressor = HuberRegressor(
    epsilon=convert_param(
        best_params_huber_regressor['epsilon'],
        'float',
        'epsilon'
    ),
    alpha=convert_param(best_params_huber_regressor['alpha'], 'float', 'alpha'),

    max_iter=convert_param(
        best_params_huber_regressor['max_iter'],
        'int',
        'max_iter'
    ),
    tol=convert_param(best_params_huber_regressor['tol'], 'float', 'tol')
)

best_params_bagging_regressor = get_best_params("BaggingRegressor")

bagging_regressor = BaggingRegressor(
    estimator=HuberRegressor(),
    n_estimators=convert_param(
        best_params_bagging_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    max_samples=convert_param(
        best_params_bagging_regressor['max_samples'],
        'float',
        'max_samples'
    ),
    max_features=convert_param(
        best_params_bagging_regressor['max_features'],
        'float',
        'max_features'
    ),
    bootstrap=convert_param(
        best_params_bagging_regressor['bootstrap'],
        'bool',
        'bootstrap'
    ),
    bootstrap_features=convert_param(
        best_params_bagging_regressor['bootstrap_features'],
        'bool',
        'bootstrap_features'
    ),
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS
)

best_params_hist_gradient_boosting_regressor = get_best_params(
    "HistGradientBoostingRegressor"
)

hist_gradient_boosting_regressor = HistGradientBoostingRegressor(
    learning_rate=convert_param(
        best_params_hist_gradient_boosting_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_iter=convert_param(
        best_params_hist_gradient_boosting_regressor['max_iter'],
        'int',
        'max_iter'
    ),
    max_depth=convert_param(
        best_params_hist_gradient_boosting_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    min_samples_leaf=convert_param(
        best_params_hist_gradient_boosting_regressor['min_samples_leaf'],
        'int',
        'min_samples_leaf'
    ),
    l2_regularization=convert_param(
        best_params_hist_gradient_boosting_regressor['l2_regularization'],
        'float',
        'l2_regularization'
    ),
    max_bins=convert_param(
        best_params_hist_gradient_boosting_regressor['max_bins'],
        'int',
        'max_bins'
    ),
    max_leaf_nodes=convert_param(
        best_params_hist_gradient_boosting_regressor['max_leaf_nodes'],
        'int',
        'max_leaf_nodes'
    ),
    tol=convert_param(
        best_params_hist_gradient_boosting_regressor['tol'],
        'float',
        'tol'
    ),
    validation_fraction=convert_param(
        best_params_hist_gradient_boosting_regressor['validation_fraction'],
        'float',
        'validation_fraction'
    ),
    loss=best_params_hist_gradient_boosting_regressor['loss'],
    random_state=RANDOM_STATE,
    interaction_cst=best_params_hist_gradient_boosting_regressor[
        'interaction_cst'
    ],
    n_iter_no_change=convert_param(
        best_params_hist_gradient_boosting_regressor['n_iter_no_change'],
        'int',
        'n_iter_no_change'
    )
)

best_params_nu_svr = get_best_params("NuSVR")

nu_svr = NuSVR(
    C=convert_param(best_params_nu_svr['C'], 'float', 'C'),
    kernel=best_params_nu_svr['kernel'],
    nu=convert_param(best_params_nu_svr['nu'], 'float', 'nu'),
    shrinking=convert_param(
        best_params_nu_svr['shrinking'],
        'bool',
        'shrinking'
    ),
    tol=convert_param(best_params_nu_svr['tol'], 'float', 'tol')
)

best_params_xgb_regressor = get_best_params("XGBoostRegressor")
best_params_xgb_regressor.pop('early_stopping_rounds', None)

xgb_regressor = XGBRegressor(
    base_score=convert_param(
        best_params_xgb_regressor['base_score'],
        'float',
        'base_score'
    ),
    booster=best_params_xgb_regressor['booster'],
    colsample_bylevel=convert_param(
        best_params_xgb_regressor['colsample_bylevel'],
        'float',
        'colsample_bylevel'
    ),
    colsample_bytree=convert_param(
        best_params_xgb_regressor['colsample_bytree'],
        'float',
        'colsample_bytree'
    ),
    gamma=convert_param(best_params_xgb_regressor['gamma'], 'float', 'gamma'),
    grow_policy=best_params_xgb_regressor['grow_policy'],
    importance_type=best_params_xgb_regressor['importance_type'],
    learning_rate=convert_param(
        best_params_xgb_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_bin=convert_param(
        best_params_xgb_regressor['max_bin'],
        'int',
        'max_bin'
    ),
    max_delta_step=convert_param(
        best_params_xgb_regressor['max_delta_step'],
        'float',
        'max_delta_step'
    ),
    max_depth=convert_param(
        best_params_xgb_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    max_leaves=convert_param(
        best_params_xgb_regressor['max_leaves'],
        'int',
        'max_leaves'
    ),
    min_child_weight=convert_param(
        best_params_xgb_regressor['min_child_weight'],
        'float',
        'min_child_weight'
    ),
    n_estimators=convert_param(
        best_params_xgb_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    num_parallel_tree=convert_param(
        best_params_xgb_regressor['num_parallel_tree'],
        'int',
        'num_parallel_tree'
    ),
    reg_alpha=convert_param(
        best_params_xgb_regressor['reg_alpha'],
        'float',
        'reg_alpha'
    ),
    reg_lambda=convert_param(
        best_params_xgb_regressor['reg_lambda'],
        'float',
        'reg_lambda'
    ),
    subsample=convert_param(
        best_params_xgb_regressor['subsample'],
        'float',
        'subsample'
    ),
    tree_method=best_params_xgb_regressor['tree_method'],
    objective='reg:squarederror',
    eval_metric='rmse',
    n_jobs=N_JOBS,
    random_state=RANDOM_STATE,
    sampling_method='uniform'
)

best_params_lgbm_regressor = get_best_params("LightGBMRegressor")

lgbm_regressor = LGBMRegressor(
    boosting_type=best_params_lgbm_regressor['boosting_type'],
    colsample_bytree=convert_param(
        best_params_lgbm_regressor['colsample_bytree'],
        'float',
        'colsample_bytree'
    ),
    importance_type=best_params_lgbm_regressor['importance_type'],
    learning_rate=convert_param(
        best_params_lgbm_regressor['learning_rate'],
        'float',
        'learning_rate'
    ),
    max_depth=convert_param(
        best_params_lgbm_regressor['max_depth'],
        'int',
        'max_depth'
    ),
    min_child_samples=convert_param(
        best_params_lgbm_regressor['min_child_samples'],
        'int',
        'min_child_samples'
    ),
    min_child_weight=convert_param(
        best_params_lgbm_regressor['min_child_weight'],
        'float',
        'min_child_weight'
    ),
    min_split_gain=convert_param(
        best_params_lgbm_regressor['min_split_gain'],
        'float',
        'min_split_gain'
    ),
    n_estimators=convert_param(
        best_params_lgbm_regressor['n_estimators'],
        'int',
        'n_estimators'
    ),
    num_leaves=convert_param(
        best_params_lgbm_regressor['num_leaves'],
        'int',
        'num_leaves'
    ),
    reg_alpha=convert_param(
        best_params_lgbm_regressor['reg_alpha'],
        'float',
        'reg_alpha'
    ),
    reg_lambda=convert_param(
        best_params_lgbm_regressor['reg_lambda'],
        'float',
        'reg_lambda'
    ),
    subsample=convert_param(
        best_params_lgbm_regressor['subsample'],
        'float',
        'subsample'
    ),
    subsample_for_bin=convert_param(
        best_params_lgbm_regressor['subsample_for_bin'],
        'int',
        'subsample_for_bin'
    ),
    subsample_freq=convert_param(
        best_params_lgbm_regressor['subsample_freq'],
        'int',
        'subsample_freq'
    ),
    n_jobs=N_JOBS,
    random_state=RANDOM_STATE,
    verbose=-1,
    objective='regression',
    metric='rmse'
)

selected_features = [
    # 'publisher_avg_clicks_transformed_scaled',
    'market_popularity_transformed_scaled',
    'category_id_encoded_scaled',
    # 'industry_encoded_scaled',
    'publisher_encoded_scaled',
    'market_id_encoded_scaled',
    'day_of_week_encoded_scaled',
    'month_encoded_scaled' #,
    # 'is_weekend_encoded_scaled',
    # 'pub_popularity_x_pub_clicks',
    # 'market_clicks_x_market_popularity'
]

X_train_full = df_train[selected_features]
X_val = df_val[selected_features]

n_trials = 500
n_splits = 5
n_repeats = 3
mlflow_experiment = "StackingRegressor_LassoLarsCV"
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
study_name = f"{mlflow_experiment}_{current_time}"
storage = f"sqlite:///{PATH}/{study_name}.db"

# Define base estimators
estimators = [
    ('linear_svr', linear_svr),
    ('huber_regressor', huber_regressor),
    ('bagging_regressor', bagging_regressor),
    ('hist_gradient_boosting', hist_gradient_boosting_regressor),
    ('nu_svr', nu_svr),
    ('xgb_regressor', xgb_regressor),
    ('lgbm_regressor', lgbm_regressor)
]

def objective(trial):
  # Tune winsorization limits
  winsorize_upper = trial.suggest_float('winsorize_upper', 0.75, 0.95)

  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-winsorize_upper)
  ))

  # Tune whether to add noise
  add_noise = trial.suggest_categorical('add_noise', [True, False])
  noise_std = 0.0

  if add_noise:
    noise_std = trial.suggest_float('noise_std', 0.001, 0.02, log=True)

  # Tune LassoLarsCV parameters
  fit_intercept = trial.suggest_categorical(
      'lasso_fit_intercept', [True, False]
  )

  max_iter = trial.suggest_int('lasso_max_iter', 500, 5000)
  max_n_alphas = trial.suggest_int('lasso_max_n_alphas', 100, 500)
  eps = trial.suggest_float('lasso_eps', 1e-6, 1e-5, log=True)
  positive = trial.suggest_categorical('lasso_positive', [True, False])

  # Initialize LassoLarsCV as final estimator
  final_estimator = LassoLarsCV(
      fit_intercept=fit_intercept,
      max_iter=max_iter,
      max_n_alphas=max_n_alphas,
      n_jobs=N_JOBS,
      eps=eps,
      positive=positive
  )

  # Repeated K-Fold Cross-Validation
  rkf = RepeatedKFold(
      n_splits=n_splits,
      n_repeats=n_repeats,
      random_state=RANDOM_STATE
  )

  rmse_list = []

  for fold_idx, (train_idx, val_idx) in enumerate(rkf.split(X_train_full)):
    X_train, X_val_fold = (
        X_train_full.iloc[train_idx],
        X_train_full.iloc[val_idx]
    )
    y_train, y_val_fold = y_train_full[train_idx], y_train_full[val_idx]

    # Add Gaussian noise to training data if selected
    if add_noise:
      noise = np.random.normal(0, noise_std, X_train.shape)

      X_train_noisy = pd.DataFrame(
          X_train + noise, columns=X_train.columns, index=X_train.index
      )
    else:
      X_train_noisy = X_train

    # Initialize StackingRegressor
    model = StackingRegressor(
        estimators=estimators,
        final_estimator=final_estimator,
        n_jobs=N_JOBS,
        cv=5,
        passthrough=False
    )

    # Fit StackingRegressor
    try:
      model.fit(X_train_noisy, y_train)
    except ValueError as e:
      print(f"Error fitting model with LassoLarsCV: {e}")
      raise optuna.TrialPruned(f"Failed to fit LassoLarsCV: {e}")

    y_pred = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    rmse_list.append(rmse)

    del X_train, X_val_fold, y_train, y_val_fold, y_pred, model, X_train_noisy

    if add_noise:
      del noise

    gc.collect()

    trial.report(np.mean(rmse_list), step=fold_idx)

    if trial.should_prune():
      raise optuna.TrialPruned()

  return np.mean(rmse_list)

# Start MLflow experiment
mlflow.set_experiment(mlflow_experiment)

with mlflow.start_run():
  study = optuna.create_study(
      study_name=study_name,
      storage=storage,
      direction="minimize",
      load_if_exists=True,
      pruner=optuna.pruners.HyperbandPruner(
          min_resource=max(1, n_splits),
          max_resource=n_splits * n_repeats,
          reduction_factor=2
      ),
      sampler=optuna.samplers.TPESampler(
          n_startup_trials=max(1, int(n_trials * 0.2)),
          seed=RANDOM_STATE,
          multivariate=True
      )
  )

  study.optimize(objective, n_trials=n_trials)
  log_optuna_best_trial_search_space(study)

  # Log Optuna storage details
  mlflow.log_param("optuna_storage", storage)
  mlflow.log_param("optuna_study_name", study_name)

  # Log experiment parameters
  mlflow.log_param("n_trials", n_trials)
  mlflow.log_param("n_splits", n_splits)
  mlflow.log_param("n_repeats", n_repeats)

  # Log selected features
  mlflow.log_param("selected_features", selected_features)

  fanova_importances = get_param_importances(
      study,
      evaluator=FanovaImportanceEvaluator(seed=RANDOM_STATE),
      target= (
          lambda t: t.value
          if t.state == optuna.trial.TrialState.COMPLETE
          else None
      ),
      normalize=True
  )

  fanova_importances = {k: round(v, 4) for k, v in fanova_importances.items()}

  for param, importance in fanova_importances.items():
    mlflow.log_param(f"fanova_{param}", importance)

  print(f'Fanova Hyperparameter Importances (rounded): {fanova_importances}')

  # Get best parameters
  best_params = study.best_params

  for param, value in best_params.items():
    mlflow.log_param(param, value)

  mlflow.log_metric("best_cv_rmse", study.best_value)

  print("Best Params:", best_params)
  print("Best CV RMSE:", study.best_value)

  # Apply best winsorization to final training and validation sets
  y_train_full = np.log1p(mstats.winsorize(
      df_train[TARGET].to_numpy(dtype=np.float64),
      limits=(0, 1-best_params['winsorize_upper'])
  ))

  y_val = np.log1p(np.clip(
      df_val[TARGET],
      0,
      np.percentile(df_train[TARGET], best_params['winsorize_upper'] * 100)
  ))

  X_final = X_train_full.copy()

  # Prepare final estimator with best parameters
  final_estimator = LassoLarsCV(
      fit_intercept=best_params['lasso_fit_intercept'],
      max_iter=best_params['lasso_max_iter'],
      max_n_alphas=best_params['lasso_max_n_alphas'],
      n_jobs=N_JOBS,
      eps=best_params['lasso_eps'],
      positive=best_params['lasso_positive']
  )

  # Train final StackingRegressor
  final_model = StackingRegressor(
      estimators=estimators,
      final_estimator=final_estimator,
      n_jobs=N_JOBS,
      cv=5,
      passthrough=False
  )

  # Add noise to final training data if selected
  if best_params['add_noise']:
    noise = np.random.normal(0, best_params['noise_std'], X_final.shape)

    X_final = pd.DataFrame(
        X_final + noise, columns=X_final.columns, index=X_final.index
    )

  X_final = X_final[sorted(X_final.columns)]
  final_model.fit(X_final, y_train_full)

  mlflow.sklearn.log_model(final_model, name="StackingRegressor")

  X_val = X_val[sorted(X_val.columns)]
  y_pred = final_model.predict(X_val)

  y_val_original, y_pred_original = evaluate_and_log_metrics(
      y_val,
      y_pred,
      prefix="val"
  )

  # SHAP analysis
  shap_analysis(final_model, X_final, model_type="kernel")

# Model Debugging

## Learning Curve

In [ ]:
# Plot learning curves to diagnose whether the model suffers from high bias
# or high variance.
train_sizes, train_scores, val_scores = learning_curve(
    final_model,
    X_final,
    y_train_full,
    cv=5,
    scoring='neg_mean_squared_error',
    train_sizes=np.linspace(0.1, 1.0, 10)
)

plt.plot(train_sizes, np.sqrt(-train_scores.mean(axis=1)), label='Train RMSE')
plt.plot(
    train_sizes,
    np.sqrt(-val_scores.mean(axis=1)),
    label='Validation RMSE'
)
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.legend()
plt.show()

## Feature Interaction Analysis

In [ ]:
# Use SHAP's interaction values to understand
# how features interact.
shap_interaction_values = explainer.shap_interaction_values(X_final)
shap.summary_plot(shap_interaction_values, X_final, plot_type="compact_dot")

## Outlier Inspection

In [ ]:
# Initialize SHAP explainer
explainer = shap.Explainer(final_model, X_final)

# Compute SHAP values for the input data
shap_values = explainer(X_final)

# Outlier Inspection: Examine instances with high SHAP values to identify
# potential outliers or anomalies driving poor generalization.
row_importance = np.abs(shap_values.values).sum(axis=1)
threshold = np.percentile(row_importance, 95)
high_impact_instances = X_final.iloc[row_importance > threshold]

# This is a DataFrame containing only those rows (instances) from X_final
# where the model's prediction was heavily influenced by the
# input features - potentially outliers or anomalies.
print(high_impact_instances.shape)

# You may decide to remove high-impact (outlier) instances from df_train to
# improve the model performance.

# Model Performance Plots

In [ ]:
residuals = y_val_original - y_pred_original

In [ ]:
# Step 1: Find experiment ID by name
experiment_name = "StackingRegressor_LassoLarsCV"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
  raise ValueError(f"No experiment found with name: {experiment_name}")

experiment_id = experiment.experiment_id

# Step 2: Search for a specific run
runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.test_rmse_original ASC"]
)

# Choose the top run
run_id = runs.iloc[0]["run_id"]

# Step 3: Load the model from the run
model_uri = f"runs:/{run_id}/StackingRegressor"
final_model = mlflow.sklearn.load_model(model_uri)

## Actual vs Predicted Plot

Key for visually assessing prediction quality.

In [ ]:
plt.figure(figsize=(8, 6))

sns.scatterplot(
    x=y_test_original,
    y=y_pred_original,
    alpha=0.7,
    color="dodgerblue",
    edgecolor="k"
)

plt.plot(
    [y_test_original.min(), y_test_original.max()],
    [y_test_original.min(), y_test_original.max()],
    'r--'
)  # 45-degree line

plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

plt.title(
    f'Actual vs. Predicted '
    f'(RMSE = {np.sqrt(mean_squared_error(y_test_original, y_pred_original)):.2f})'
)

plt.grid(True)
plt.tight_layout()
plt.show()

## Residual Plot (Residuals vs Predicted)

Crucial for checking heteroscedasticity and non-linearity.

In [ ]:
# Plot
plt.figure(figsize=(8, 6))

sns.scatterplot(
    x=y_pred_original,
    y=residuals,
    color='darkorange',
    edgecolor='k',
    alpha=0.7
)

plt.axhline(0, linestyle='--', color='red')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residuals vs. Predicted Values')
plt.grid(True)
plt.tight_layout()
plt.show()

## Q-Q Plot (Quantile-Quantile Plot)

Best way to check if residuals are normally distributed.

In [ ]:
plt.figure(figsize=(8, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals")
plt.grid(True)
plt.tight_layout()
plt.show()

## Bar Plot of Error Metrics

Clear summary of model performance across metrics.

In [ ]:
# Define actual and predicted values
y_train = df_train[TARGET]
X_train = df_train[selected_features]
y_test = y_test_original
y_pred = y_pred_original

# Calculate metrics for train
mae_train = mean_absolute_error(y_train, final_model.predict(X_train))
rmse_train = np.sqrt(mean_squared_error(y_train, final_model.predict(X_train)))
r2_train = r2_score(y_train, final_model.predict(X_train))

# Calculate metrics for test
mae_test = mean_absolute_error(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
r2_test = r2_score(y_test, y_pred)

# Store metrics
metrics = ['MAE', 'RMSE', 'R2']
train_scores = [mae_train, rmse_train, r2_train]
test_scores = [mae_test, rmse_test, r2_test]

# Compute percentage differences
percentage_diff = [
    (
        ((test - train) / train * 100)
        if metric != 'R2'
        else ((test - train) / abs(train) * 100)
    )
    for train, test, metric in zip(train_scores, test_scores, metrics)
]

# Create a DataFrame for display
df_metrics = pd.DataFrame({
    'Metric': metrics,
    'Train': train_scores,
    'Test': test_scores,
    'Test vs Train % Diff': [f"{diff:+.2f}%" for diff in percentage_diff]
})

# Print nicely
print(df_metrics.to_string(index=False))

# Plot
x = np.arange(len(metrics))
width = 0.35

plt.figure(figsize=(8, 6))
plt.bar(x - width/2, train_scores, width, label='Train', color='skyblue')
plt.bar(x + width/2, test_scores, width, label='Test', color='salmon')

plt.xticks(x, metrics)
plt.ylabel('Score')
plt.title('Error Metrics Comparison (Train vs Test)')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Partial Dependence Plots (PDP)

Great for model interpretability, showing marginal effects of features.

In [ ]:
X_val = df_val[selected_features]

# Plot PDP for one or more features
features = selected_features
PartialDependenceDisplay.from_estimator(final_model, X_val, features)